In [5]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt

from qiskit import Aer
from tqdm.notebook import tqdm
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *

%matplotlib notebook
#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Digits data

In [6]:
digits = load_digits()
zero_idx = (digits.target == 0)
one_idx = (digits.target == 1)
two_idx = (digits.target == 2)

In [7]:
zeros = digits.data[zero_idx]
ones =  digits.data[one_idx]
twos = digits.data[two_idx]
#threes = digits.data[three_idx]

x = np.concatenate((zeros, ones, twos))
x = x - np.mean(x)
x = x/np.max(np.abs(x))

y = np.zeros((x.shape[0], 3))
y[:len(zeros), 0] = 1
y[len(zeros):(len(zeros)+len(ones)), 1] = 1
y[(len(zeros)+len(ones)):, 2] = 1
#y[(len(zeros)+len(ones) + len(twos)):, 3] = 1
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [8]:
np.random.seed(42)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=100)
print(x_train.shape)

(100, 64)


### Network

In [12]:
np.random.seed(42)
optimizer = Adam(lr=1e-1)
backend = Aer.get_backend('qasm_simulator')

layer1 = Dense(n_features=64, 
               n_targets=3, 
               scale = 2*np.pi, 
               activation = Sigmoid())

layer2 = QLayer(n_qubits=3, 
                n_features=3, 
                n_targets=3, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                sampler = Parity(),
                reps=2, 
                scale=2*np.pi, 
                backend=backend, 
                shots=10000)

layer3 = QLayer(n_qubits=3, 
                n_features=3, 
                n_targets=3, 
                encoder=Encoder(), 
                ansatz=Ansatz(), 
                sampler = Parity(),
                reps=2, 
                scale=1, 
                backend=backend, 
                shots=10000)

layers =[layer1, layer2, layer3]
network1 = NeuralNetwork(layers, optimizer)

### Training

In [13]:
network1.train(x_train, y_train, epochs=100, verbose=True)
saver(network1, data_path("digits_hybrid_layers_3"))

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 0, loss: 0.30462865696666663
epoch: 1, loss: 0.24954863146666667
epoch: 2, loss: 0.24263174143333335
epoch: 3, loss: 0.23310073246666665
epoch: 4, loss: 0.20952214843333336
epoch: 5, loss: 0.21182990286666667
epoch: 6, loss: 0.19095355709999998
epoch: 7, loss: 0.170975401
epoch: 8, loss: 0.16588673326666664
epoch: 9, loss: 0.16837166916666668
epoch: 10, loss: 0.1684513058
epoch: 11, loss: 0.16555824213333334
epoch: 12, loss: 0.15827549616666667
epoch: 13, loss: 0.14995201323333332
epoch: 14, loss: 0.14457552640000002
epoch: 15, loss: 0.14023230406666665
epoch: 16, loss: 0.1365110053666667
epoch: 17, loss: 0.13293838446666664
epoch: 18, loss: 0.12810237359999999
epoch: 19, loss: 0.12338523203333333
epoch: 20, loss: 0.11696661520000001
epoch: 21, loss: 0.11355625979999999
epoch: 22, loss: 0.1092122846
epoch: 23, loss: 0.10638719793333333
epoch: 24, loss: 0.1045068561
epoch: 25, loss: 0.10149031743333334
epoch: 26, loss: 0.0983651562
epoch: 27, loss: 0.09435572613333333
epoch: 28, 

QiskitError: 'Keyboard interrupt in parallel_map.'

In [9]:
np.random.seed(42)
network2 = sequential_dnn(dim=[64, 3, 3, 3], lr=0.1)

In [10]:
network2.train(x_train, y_train, epochs=100, verbose=True)

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 0, loss: 0.30220710574494536
epoch: 1, loss: 0.26422636183218345
epoch: 2, loss: 0.24261752690170776
epoch: 3, loss: 0.23085094554934416
epoch: 4, loss: 0.225382103410018
epoch: 5, loss: 0.22320550500979267
epoch: 6, loss: 0.22269764992066318
epoch: 7, loss: 0.22298292965556954
epoch: 8, loss: 0.22357171800858514
epoch: 9, loss: 0.2241767279069117
epoch: 10, loss: 0.22458833637124276
epoch: 11, loss: 0.2245875987815118
epoch: 12, loss: 0.22391248734998082
epoch: 13, loss: 0.22247215626486522
epoch: 14, loss: 0.2203928702252851
epoch: 15, loss: 0.21794901638389413
epoch: 16, loss: 0.21503225299271586
epoch: 17, loss: 0.2118592624563914
epoch: 18, loss: 0.20862980069887777
epoch: 19, loss: 0.2049427487035639
epoch: 20, loss: 0.20069323854366894
epoch: 21, loss: 0.19623932196197005
epoch: 22, loss: 0.19140340073600462
epoch: 23, loss: 0.18580223014780936
epoch: 24, loss: 0.17944734719059446
epoch: 25, loss: 0.17328156229545996
epoch: 26, loss: 0.167149528551041
epoch: 27, loss: 0.1

In [11]:
y_pred = np.round(network.predict(x_train))
print(np.mean(y_pred == y_train))

NameError: name 'network' is not defined

In [ ]:
y_pred = np.round(network2.predict(x_train))
print(np.mean(y_pred == y_train))